In [1]:
from sqlalchemy import create_engine
import pymysql
import pandas as pd
import numpy as np

import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt
import squarify
import pygal
import plotly.express as px

### Updating existing Metacritic dataset with missing values
Current imported dataset [imported from Kaggle : Video Game Sales with Ratings](https://www.kaggle.com/rush4ratio/video-game-sales-with-ratings) miss values :
- missing several Critic scores
- missing recent games post 2016

Adding missing values with ad hoc dictionary

In [2]:
df_7K_all_consoles = pd.read_csv(r'../datasets/metacritic_6900_games_22_Dec_2016.csv')

df_7K_all_consoles

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16714,Samurai Warriors: Sanada Maru,PS3,2016.0,Action,Tecmo Koei,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16715,LMA Manager 2007,X360,2006.0,Sports,Codemasters,0.00,0.01,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16716,Haitaka no Psychedelica,PSV,2016.0,Adventure,Idea Factory,0.00,0.00,0.01,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16717,Spirits & Spells,GBA,2003.0,Platform,Wanadoo,0.01,0.00,0.00,0.00,0.01,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df_7K_all_consoles_updated = df_7K_all_consoles.copy()

In [4]:
#Adding new data
# Creating Dict with missing values
# for looping
new_row = [{'Name':'God of War IV',
                           'Platform':'PS4',
                           'Year_of_Release':2018, 
                           'Genre':"Beat'em All",
                          'Publisher':'Santa Monica Studio',
                          'Global_Sales':11,
                          'Critic_Score':94,
                           'User_Score':9.1
                          },
                          {'Name':'Final Fantasy XV',
                           'Platform': 'PS4',
                           'Year_of_Release':2016, 
                           'Genre':"JRPG",
                          'Publisher':'Square Enix',
                          'Global_Sales':9,
                          'Critic_Score':81,
                           'User_Score':7.6
                          },
                          {'Name':'Final Fantasy VII Remake',
                           'Platform': 'PS4',
                           'Year_of_Release':2020, 
                           'Genre':"JRPG",
                          'Publisher':'Square Enix',
                          'Global_Sales':4,
                          'Critic_Score':88,
                           'User_Score':8.0
                          },
                          {'Name':'Spiderman',
                           'Platform': 'PS4',
                           'Year_of_Release':2018, 
                           'Genre':"JRPG",
                          'Publisher':'Square Enix',
                          'Global_Sales':13.5,
                          'Critic_Score':87,
                           'User_Score':8.6
                          },
                          {'Name':"Assassin's Creed Origins",
                           'Platform': 'PS4',
                           'Year_of_Release':2017, 
                           'Genre':"Action Adventure|Open-World",
                          'Publisher':'Ubisoft',
                          'Global_Sales':11,
                          'Critic_Score':81,
                           'User_Score':7.2
                          },
                          {'Name':"Assassin's Creed Odyssey",
                           'Platform': 'PC',
                           'Year_of_Release':2018, 
                           'Genre':"Action Adventure|Open-World",
                          'Publisher':'Ubisoft',
                          'Global_Sales':11,
                          'Critic_Score':86,
                           'User_Score':5.9
                          },
                          {'Name':'Star Wars Jedi Knight II: Jedi Outcast',
                           'Platform': 'PC',
                           'Year_of_Release':2002, 
                           'Genre':"Action-Adventure",
                          'Publisher':'Lucas Arts',
                          'Global_Sales':0.93,
                          'Critic_Score':89,
                           'User_Score':7.6
                          },
                          {'Name':'Star Wars Episode I Racer',
                           'Platform': 'PC',
                           'Year_of_Release':1999, 
                           'Genre':"Racing",
                          'Publisher':'Lucas Arts',
                          'Global_Sales':3.12,
                          'Critic_Score':85,
                           'User_Score':8.6
                          },
                          {'Name':"Baldur's Gate",
                           'Platform': 'PC',
                           'Year_of_Release':1998, 
                           'Genre':"RPG",
                          'Publisher':'BioWare',
                          'Global_Sales':3,
                          'Critic_Score':91,
                           'User_Score':8.9
                          },
                          {'Name':"Baldur's Gate II: Shadows of Amn",
                           'Platform': 'PC',
                           'Year_of_Release':1998, 
                           'Genre':"RPG",
                          'Publisher':'BioWare',
                          'Global_Sales':2.5,
                          'Critic_Score':95,
                           'User_Score':9.1
                          },
                          {'Name':"The Curse of Monkey Island",
                           'Platform': 'PC',
                           'Year_of_Release':1997, 
                           'Genre':"Point&Click",
                          'Publisher':'Lucas Arts',
                          'Global_Sales':0.9,
                          'Critic_Score':89,
                           'User_Score':9.0
                          },
                          {'Name':"Escape From Monkey Island",
                           'Platform': 'PC',
                           'Year_of_Release':2001, 
                           'Genre':"Point&Click",
                          'Publisher':'Lucas Arts',
                          'Global_Sales':0.1,
                          'Critic_Score':84,
                           'User_Score':8.2
                          },
                          {'Name':"The Secrets of Monkey Island",
                           'Platform': 'PC',
                           'Year_of_Release':1990, 
                           'Genre':"Point&Click",
                          'Publisher':'Lucas Arts',
                          'Global_Sales':0.1,
                          'Critic_Score':86,
                           'User_Score':8.9
                          },
                          {'Name':"Little Big Adventure",
                           'Platform': 'PC',
                           'Year_of_Release':1994, 
                           'Genre':"Action-Adventure|Platformer",
                          'Publisher':'Adeline Software',
                          'Global_Sales':0.7,
                          'Critic_Score':85,
                           'User_Score':8.7
                          },
                         {'Name':"Little Big Adventure 2",
                           'Platform': 'PC',
                           'Year_of_Release':1997, 
                           'Genre':"Action-Adventure|Platformer",
                          'Publisher':'Adeline Software',
                          'Global_Sales':0.5,
                          'Critic_Score':85,
                           'User_Score':8.8
                          },
                          {'Name':"Heroes of Might and Magic IV",
                           'Platform': 'PC',
                           'Year_of_Release':2002,
                           'Genre':"STR",
                          'Publisher':'The 3DO Company',
                          'Global_Sales':2.5,
                          'Critic_Score':84,
                           'User_Score':8.1
                          },
                          {'Name':"Heroes of Might and Magic III",
                           'Platform': 'PC',
                           'Year_of_Release':1999, 
                           'Genre':"STR",
                          'Publisher':'The 3DO Company',
                          'Global_Sales':3,
                          'Critic_Score':87,
                           'User_Score':9.1
                          },
                          {'Name':"Heroes of Might and Magic V",
                           'Platform': 'PC',
                           'Year_of_Release':1998, 
                           'Genre':"STR",
                          'Publisher':'The 3DO Company',
                          'Global_Sales':2.5,
                          'Critic_Score':95,
                           'User_Score':9.1
                          },
                          {'Name':'Heroes of Might and Magic VII',
                           'Platform': 'PC',
                           'Year_of_Release':2015, 
                           'Genre':"STR",
                          'Publisher':'Ubisoft',
                          'Global_Sales':'NaN',
                          'Critic_Score':67,
                           'User_Score':5.0
                                                 },
                          
                          {'Name':'Heroes of the Storm',
                           'Platform': 'PC',
                           'Year_of_Release':2016, 
                           'Genre':"MOBA",
                          'Publisher':'Blizzard',
                          'Global_Sales':11,
                          'Critic_Score':81,
                           'User_Score':7.6
                          },
                          {'Name':'Star Wars Battlefront II',
                           'Platform': 'PS4',
                           'Year_of_Release':2017, 
                           'Genre':"FPS|Space Shooter",
                          'Publisher':'Electronic Arts',
                          'Global_Sales':10,
                          'Critic_Score':78,
                           'User_Score':8.6
                          },
                                {'Name':'Star Wars: Galactic Battlegrounds',
                           'Platform': 'PC',
                           'Year_of_Release':2001, 
                           'Genre':"STR",
                          'Publisher':'Lucas Arts',
                          'Global_Sales':0.5,
                          'Critic_Score':75,
                           'User_Score':7.9
                          },
                               {'Name':'Carmageddon',
                           'Platform': 'PC',
                           'Year_of_Release':1997, 
                           'Genre':"Racing|Beat'em All",
                          'Publisher':'SCi',
                          'Global_Sales':0.3,
                          'Critic_Score':85,
                           'User_Score':8.4
                          },
                               {'Name':'Carmageddon II: Carpocalypse Now"',
                           'Platform': 'PC',
                           'Year_of_Release':1998, 
                           'Genre':"Racing|Beat'em All",
                          'Publisher':'SCi',
                          'Global_Sales':0.2,
                          'Critic_Score':85,
                           'User_Score':8.4
                          },
           
                               {'Name':'The Legend of Zelda: Breath to the Wild',
                           'Platform': 'Switch',
                           'Year_of_Release':2017, 
                           'Genre':"Action-Adventure|Open World",
                          'Publisher':'Nintendo',
                          'Global_Sales':18,
                          'Critic_Score':97,
                           'User_Score':8.6
                          },
                                {'Name':'Mario Kart: Double Dash',
                           'Platform': 'GameCube',
                           'Year_of_Release':2003, 
                           'Genre':"Racing",
                          'Publisher':'Nintendo',
                          'Global_Sales':7.5,
                          'Critic_Score':87,
                           'User_Score':8.6
                          },
                                {'Name':'Mario Kart Deluxe',
                           'Platform': 'Switch',
                           'Year_of_Release':2018, 
                           'Genre':"Racing",
                          'Publisher':'Nintendo',
                          'Global_Sales':24.8,
                          'Critic_Score':92,
                           'User_Score':8.6
                          },
                      {'Name':'Command & Conquer: Generals',
                           'Platform': 'PC',
                           'Year_of_Release':2003, 
                           'Genre':"STR",
                          'Publisher':'Westwood',
                          'Global_Sales':0.4,
                          'Critic_Score':84,
                           'User_Score':8.4
                          },
                                 {'Name':'Kingdom Hearts III',
                           'Platform': 'PS4',
                           'Year_of_Release':2018, 
                           'Genre':"Action|JRPG",
                          'Publisher':'Square Enix',
                          'Global_Sales':7,
                          'Critic_Score':83,
                           'User_Score':8.1
                          },
                                 {'Name':'Red Dead Redemption 2',
                           'Platform': 'PS4',
                           'Year_of_Release':2018, 
                                  'Genre':"Action-Adventure|Open World",
                          'Publisher':'Rockstar',
                          'Global_Sales':30,
                          'Critic_Score':97,
                           'User_Score':8.2
                          },
                                 {'Name':'Grand Theft Auto',
                           'Platform': 'PC',
                           'Year_of_Release':1998, 
                           'Genre':"Action-Adventure|Open World",
                          'Publisher':'Rockstar',
                          'Global_Sales':0.4,
                          'Critic_Score':69,
                           'User_Score':8.1
                          },
                                 {'Name':'Horizon Zero Dawn',
                           'Platform': 'PS4',
                           'Year_of_Release':2017, 
                           'Genre':"Action-Adventure|Open World",
                          'Publisher':'Guerrilla',
                          'Global_Sales':11,
                          'Critic_Score':89,
                           'User_Score':8.4
                          },
                                 {'Name':'Grim Fandango',
                           'Platform': 'PC',
                           'Year_of_Release':1998, 
                           'Genre':"Point&Click",
                          'Publisher':'Lucas Arts',
                          'Global_Sales':0.5,
                          'Critic_Score':94,
                           'User_Score':9.1
                          },
                                 {'Name':'WipeOut 2097',
                           'Platform': 'PS1',
                           'Year_of_Release':1996, 
                           'Genre':"Racing",
                          'Publisher':'	Psygnosis',
                          'Global_Sales':0.4,
                          'Critic_Score':93,
                           'User_Score':8.6
                          },
          ]

In [5]:
#for looping
for i in range(len(new_row)):
    print(new_row[i])
#     df_7K_all_consoles_updated = df_7K_all_consoles_updated.append(i, ignore_index=True)

{'Name': 'God of War IV', 'Platform': 'PS4', 'Year_of_Release': 2018, 'Genre': "Beat'em All", 'Publisher': 'Santa Monica Studio', 'Global_Sales': 11, 'Critic_Score': 94, 'User_Score': 9.1}
{'Name': 'Final Fantasy XV', 'Platform': 'PS4', 'Year_of_Release': 2016, 'Genre': 'JRPG', 'Publisher': 'Square Enix', 'Global_Sales': 9, 'Critic_Score': 81, 'User_Score': 7.6}
{'Name': 'Final Fantasy VII Remake', 'Platform': 'PS4', 'Year_of_Release': 2020, 'Genre': 'JRPG', 'Publisher': 'Square Enix', 'Global_Sales': 4, 'Critic_Score': 88, 'User_Score': 8.0}
{'Name': 'Spiderman', 'Platform': 'PS4', 'Year_of_Release': 2018, 'Genre': 'JRPG', 'Publisher': 'Square Enix', 'Global_Sales': 13.5, 'Critic_Score': 87, 'User_Score': 8.6}
{'Name': "Assassin's Creed Origins", 'Platform': 'PS4', 'Year_of_Release': 2017, 'Genre': 'Action Adventure|Open-World', 'Publisher': 'Ubisoft', 'Global_Sales': 11, 'Critic_Score': 81, 'User_Score': 7.2}
{'Name': "Assassin's Creed Odyssey", 'Platform': 'PC', 'Year_of_Release': 2

In [6]:
#for loop with pandas.append() method
for i in range(len(new_row)):
    df_7K_all_consoles_updated = df_7K_all_consoles_updated.append(new_row[i], ignore_index=True)

In [7]:
df_7K_all_consoles_updated.tail(50)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
16703,The Longest 5 Minutes,PSV,2016.0,Action,Nippon Ichi Software,0.00,0.00,0.01,0.0,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16704,Mezase!! Tsuri Master DS,DS,2009.0,Sports,Hudson Soft,0.00,0.00,0.01,0.0,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16705,Eiyuu Densetsu: Sora no Kiseki Material Collec...,PSP,2007.0,Role-Playing,Falcom Corporation,0.00,0.00,0.01,0.0,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16706,STORM: Frontline Nation,PC,2011.0,Strategy,Unknown,0.00,0.01,0.00,0.0,0.01,60.0,12.0,7.2,13.0,SimBin,E10+
16707,Strawberry Nauts,PSV,2016.0,Adventure,Unknown,0.00,0.00,0.01,0.0,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16708,Plushees,DS,2008.0,Simulation,Destineer,0.01,0.00,0.00,0.0,0.01,NaN,NaN,tbd,NaN,Big John Games,E
16709,15 Days,PC,2009.0,Adventure,DTP Entertainment,0.00,0.01,0.00,0.0,0.01,63.0,6.0,5.8,8.0,DTP Entertainment,NaN
16710,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.0,0.01,NaN,NaN,tbd,NaN,Atari,T
16711,Aiyoku no Eustia,PSV,2014.0,Misc,dramatic create,0.00,0.00,0.01,0.0,0.01,NaN,NaN,NaN,NaN,NaN,NaN
16712,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.0,0.01,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df_7K_all_consoles.dropna(subset=['Name'], axis=0, inplace=True)

In [9]:
df_7K_all_consoles[df_7K_all_consoles['Name'].str.contains("Final F")]

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
65,Final Fantasy VII,PS,1997.0,Role-Playing,Sony Computer Entertainment,3.01,2.47,3.28,0.96,9.72,92.0,20.0,9.2,1282.0,SquareSoft,T
84,Final Fantasy X,PS2,2001.0,Role-Playing,Sony Computer Entertainment,2.91,2.07,2.73,0.33,8.05,92.0,53.0,8.7,1056.0,SquareSoft,T
88,Final Fantasy VIII,PS,1999.0,Role-Playing,SquareSoft,2.28,1.72,3.63,0.23,7.86,90.0,24.0,8.6,644.0,SquareSoft,T
148,Final Fantasy XII,PS2,2006.0,Role-Playing,Square Enix,1.88,0.00,2.33,1.74,5.95,92.0,64.0,7.6,972.0,Square Enix,T
173,Final Fantasy XIII,PS3,2009.0,Role-Playing,Square Enix,1.74,1.21,1.87,0.51,5.33,83.0,83.0,7.3,2483.0,Square Enix,T
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12869,Final Fight: Streetwise,PS2,2006.0,Action,Capcom,0.03,0.02,0.00,0.01,0.05,43.0,36.0,6.2,11.0,Capcom,M
15048,Final Fantasy XIV: Heavensward,PC,2015.0,Action,Square Enix,0.01,0.01,0.00,0.00,0.02,86.0,14.0,7.7,255.0,Square Enix,T
15054,Final Fantasy XIV: Heavensward,PS3,2015.0,Action,Square Enix,0.01,0.00,0.01,0.00,0.02,NaN,NaN,8.2,72.0,Square Enix,T
15830,Final Fight: Streetwise,XB,2006.0,Action,Capcom,0.01,0.00,0.00,0.00,0.02,42.0,29.0,4,4.0,Secret Level,M


In [10]:
# Updating metascore & platform on corrupted values

#updating SONIC metascore
df_7K_all_consoles_updated['Critic_Score'][257]= 85
df_7K_all_consoles_updated['Critic_Score'][145]= 87
df_7K_all_consoles_updated['Critic_Score'][996]= 86
df_7K_all_consoles_updated['Critic_Score'][960]= 85

#updating WatchDogsmetascore
df_7K_all_consoles_updated['Critic_Score'][1021]= 80 #WatchDogsmetascore
df_7K_all_consoles_updated['User_Score'][1021]= 6.4

#updating ZELDA metascore
df_7K_all_consoles_updated['Critic_Score'][93]= 99 #Ocarina
df_7K_all_consoles_updated['User_Score'][93]= 9.1

#updating Okami metascore
df_7K_all_consoles_updated['Critic_Score'][3208]= 93 #Okami
df_7K_all_consoles_updated['User_Score'][3208]= 8.1

df_7K_all_consoles_updated['Critic_Score'][232]= 95 #Link to the Past
df_7K_all_consoles_updated['User_Score'][232]= 8.9

df_7K_all_consoles_updated['Critic_Score'][322]= 87 #Links'Awakening
df_7K_all_consoles_updated['User_Score'][322]= 8.4

#updating Tomb Raider metascore
df_7K_all_consoles_updated['Critic_Score'][646]= 87 #Reboot
df_7K_all_consoles_updated['User_Score'][646]= 8.3

df_7K_all_consoles_updated['Critic_Score'][2204]= 88 #Rise
df_7K_all_consoles_updated['User_Score'][2204]= 8.1

#updating Mario Party  metascore
df_7K_all_consoles_updated['Critic_Score'][906]= 74 #MP3
df_7K_all_consoles_updated['User_Score'][906]= 8.2

df_7K_all_consoles_updated['Critic_Score'][45]= 94 #M64
df_7K_all_consoles_updated['User_Score'][45]= 9.2

df_7K_all_consoles_updated['Critic_Score'][63]= 83 #MK64
df_7K_all_consoles_updated['User_Score'][45]= 8.6

df_7K_all_consoles_updated['Critic_Score'][76]= 83 #MKSNES
df_7K_all_consoles_updated['User_Score'][76]= 8.6

df_7K_all_consoles_updated['Critic_Score'][112]= 87 #MKGC
df_7K_all_consoles_updated['User_Score'][112]= 8.6

#updating C&C metascore
df_7K_all_consoles_updated['Platform'][4847]= 'PC' #C&C1
df_7K_all_consoles_updated['Critic_Score'][4847]= 94 #C&C1
df_7K_all_consoles_updated['User_Score'][4847]= 8.9

df_7K_all_consoles_updated['Critic_Score'][510]= 90 #Red Alert
df_7K_all_consoles_updated['User_Score'][510]= 8.8

df_7K_all_consoles_updated['Critic_Score'][1207]= 87 #C&C2
df_7K_all_consoles_updated['User_Score'][1207]= 8.3

df_7K_all_consoles_updated['Critic_Score'][12205]= 85 #C&C3
df_7K_all_consoles_updated['User_Score'][12205]= 8.0

df_7K_all_consoles_updated['Critic_Score'][1461]= 84 #Red Alert 2
df_7K_all_consoles_updated['User_Score'][1461]= 8.9


df_7K_all_consoles_updated['Critic_Score'][3026]= 82 #Red Alert 3
df_7K_all_consoles_updated['User_Score'][3026]= 6.9

df_7K_all_consoles_updated['Platform'][15745]= 'PC' #Worms 1
df_7K_all_consoles_updated['Platform'][1806]= 'PC' #Watch2
df_7K_all_consoles_updated['Platform'][16267]= 'PC' #Worms WMD

<ipython-input-10-ed52ba557388>:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-10-ed52ba557388>:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-10-ed52ba557388>:6: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-10-ed52ba557388>:7: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-d

In [11]:
df_7K_all_consoles_updated

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16748,Red Dead Redemption 2,PS4,2018.0,Action-Adventure|Open World,Rockstar,NaN,NaN,NaN,NaN,30,97.0,NaN,8.2,NaN,NaN,NaN
16749,Grand Theft Auto,PC,1998.0,Action-Adventure|Open World,Rockstar,NaN,NaN,NaN,NaN,0.4,69.0,NaN,8.1,NaN,NaN,NaN
16750,Horizon Zero Dawn,PS4,2017.0,Action-Adventure|Open World,Guerrilla,NaN,NaN,NaN,NaN,11,89.0,NaN,8.4,NaN,NaN,NaN
16751,Grim Fandango,PC,1998.0,Point&Click,Lucas Arts,NaN,NaN,NaN,NaN,0.5,94.0,NaN,9.1,NaN,NaN,NaN


In [12]:
#Exporting

In [13]:
df_7K_all_consoles_updated.to_csv('../datasets/metacritic_6900_games_22_Dec_2016_updated.csv', index=False)